## Retrieval 

Get the relevant data from document

In [ ]:
!pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain langchain-deepseek

## Part 1 Overview 

In [ ]:
import bs4 
from langchain import hub 
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.documents import Document
from langchain_community.vectorstores import Chroma, InMemoryVectorStore
from langchain_core.output_parsers import StrOutputParser 
from langchain_core.runnables import RunnablePassthrough 
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_deepseek import ChatDeepSeek
from langgraph.graph import START, StateGraph
import os
from typing_extensions import List, TypedDict

In [ ]:
import os, getpass 
from dotenv import load_dotenv
load_dotenv()


def _set_env(var:str) :
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}")
_set_env("DEEPSEEK_API_KEY")
_set_env("OPENAI_API_KEY")


_set_env("LANGSMITH_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "pr-weary-bull-48"

In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
vector_store = InMemoryVectorStore(embeddings)


In [ ]:
# Indexing 
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2024-11-28-reward-hacking/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)
assert len(docs) == 1

print(f"Total characters: {len(docs[0].page_content)}")
print(f"Split blog post into {len(all_splits)} sub-documents.")


In [ ]:
# Indexing 
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2024-11-28-reward-hacking/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)
assert len(docs) == 1
# print(f"Total characters: {len(docs[0].page_content)}")


In [ ]:
# Query Analsys 
total_documents = len(all_splits)
third = total_documents // 3


for i , document in enumerate (all_splits):
    if i<third :
        document.metadata["section"] = "begining"
    elif i < 2 * third:
        document.metadata["section"] = "middle"
    else:
        document.metadata["section"] = "end"



In [ ]:
print(docs[0].page_content[:500])

In [ ]:

# Index Chunks 

_ = vector_store.add_documents(documents=all_splits)

In [ ]:
# Search using Query 
from typing import Literal 

from typing_extensions import Annotated 

class Search(TypedDict):
    """Search Query"""
    query: Annotated[str, ..., "Search Query to run."]
    section: Annotated[
    Literal ["beginning", "middle", "end"],
    ...,
    "Section to Query,",
    ]


In [ ]:
prompt = hub.pull("rlm/rag-prompt")

example_message = prompt.invoke({
    "context" : "this is sai", 
    "question" : "who are you?"
}).to_messages()

print(f"{example_message[0].content}")
# llm = ChatDeepSeek(
#     model="deepseek-chat",
#     temperature=0,
#     max_tokens=None,
#     timeout=None,
#     max_retries=2,
# )

llm = ChatOpenAI(model="gpt-4o-mini")
example_message

In [ ]:
# Custom RAG Prompt

from langchain_core.prompts import PromptTemplate

template="""Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three to five sentences maximum and keep the answer as concise as possible.
Always say "thanks for asking!" at the end of the answer.

{context}

Question: {question}

Helpful Answer:
"""

custom_rag_prompt = PromptTemplate.from_template(template)
example_message = custom_rag_prompt.invoke({
    "context" : "this is sai", 
    "question" : "who are you?"
}).to_messages()
example_message

In [ ]:
# Define State for application 
class State (TypedDict):
    question:str
    query: Search
    context: List[Document]
    answer: str


# Define application steps 
def retrieve_old(state:State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return {"context": retrieved_docs}

def retrieve(state:State):
    query = state["query"]
    retrieved_docs = vector_store.similarity_search(
        query["query"],
        filter = lambda doc:doc.metadata.get("section") == query["section"],
    )
    return {"context" :retrieved_docs }

def analyze_query(state: State):
    strutured_llm = llm.with_structured_output(Search)
    query = strutured_llm.invoke(state["question"])
    return {"query": query}

def generate(state:State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    msgs = custom_rag_prompt.invoke({
        "question": state["question"],
        "context": docs_content
    })

    response = llm.invoke(msgs)
    return {"answer": response.content}


In [ ]:
# Compile application  and test 

graph_builder = StateGraph(State).add_sequence([analyze_query, retrieve, generate])
graph_builder.add_edge(START, "analyze_query")
graph=graph_builder.compile()

response = graph.invoke({
    "question": "What is Reward Hacking and it's examples?"
})

print(response["answer"])

In [ ]:
from IPython.display import Image, display
display(Image(graph.get_graph().draw_mermaid_png()))



In [ ]:
# Stream steps 

for step in graph.stream({"question": "Reward Hacking"}, stream_mode="updates"):
    print(f"{step}\n\n---------------\n")

In [ ]:
#Async Invokations 

response =await  graph.ainvoke({
    "question":"Reward Hacking"
})

print(f"{response["answer"]}")




In [ ]:
# async  stream message
async for step in graph.astream({"question" :"Reward Hacking"}, stream_mode="updates"):
    print(f"{step}\n\n--------\n")

In [ ]:
# async stream tokens
async for message, metadata in graph.astream({"question" :"Reward Hacking?"}, stream_mode="messages"):
    print(message.content, end="|")

In [ ]:
from IPython.display import Image, display
display(Image(graph.get_graph().draw_mermaid_png()))



In [ ]:
response = graph.invoke({
    "question": "What is does end of section says about Reward Hacking and it's examples?"
})
print(f'Context: {response["context"]}\n\n')
print(f'Answer: {response["answer"]}')


for step in graph.stream(
    {
        "question": "What is does end of section says about Reward Hacking and it's examples?"
    }, stream_mode="updates"
):
    print(f"{step}\n\n ----------\n")

In [ ]:
# Stream tokens 

for message, metadata in graph.stream(
    {
        "question" : "Reward Hacking"
    },
    stream_mode ="messages"
):
    print(message.content, end="|")

In [ ]:
response = graph.invoke({
    "question": "Reward Hacking examples?"
})
print(f'Context: {response["context"]}\n\n')
print(f'Answer: {response["answer"]}')